# RouterChains - Cadeias de roteamento

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from dotenv import load_dotenv # lib para acessar .env e chave API da Gemini

from langchain.chains.llm import LLMChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

load_dotenv()

True

In [2]:
fis_template = ChatPromptTemplate.from_template("""Você é um professor de física muito inteligente.
Você é ótimo em responder perguntas sobre física de forma concisa
e fácil de entender.
Quando você não sabe a resposta para uma pergunta, você admite
que não sabe.

Aqui está uma pergunta: {input}""")

mat_template = ChatPromptTemplate.from_template("""Você é um matemático muito bom.
Você é ótimo em responder perguntas de matemática.
Você é tão bom porque consegue decompor
problemas difíceis em suas partes componentes,
responder às partes componentes e depois juntá-las
para responder à pergunta mais ampla.

Aqui está uma pergunta: {input}""")

hist_template = ChatPromptTemplate.from_template("""Você é um historiador muito bom.
Você tem um excelente conhecimento e compreensão de pessoas,
eventos e contextos de uma variedade de períodos históricos.
Você tem a capacidade de pensar, refletir, debater, discutir e
avaliar o passado. Você tem respeito pela evidência histórica
e a capacidade de usá-la para apoiar suas explicações
e julgamentos.

Aqui está uma pergunta: {input}""")

In [3]:
prompt_infos= [
    {'name': 'Fisica',
    'description': 'Ideal para responder perguntas de física',
    'prompt_template': fis_template},
    {'name': 'Matematica',
    'description': 'Ideal para responder perguntas de matemática',
    'prompt_template': mat_template},
    {'name': 'Historia',
    'description': 'Ideal para responder perguntas de história',
    'prompt_template': hist_template}
    ]

In [8]:
chat = ChatGoogleGenerativeAI(model='gemini-1.5-flash')

#Ciando chains de destino

chains_destino = {}
for info in prompt_infos:
    chain = LLMChain(llm=chat, prompt=info['prompt_template'], verbose=True)
    chains_destino[info['name']] = chain


chains_destino

{'Fisica': LLMChain(verbose=True, prompt=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='Você é um professor de física muito inteligente.\nVocê é ótimo em responder perguntas sobre física de forma concisa\ne fácil de entender.\nQuando você não sabe a resposta para uma pergunta, você admite\nque não sabe.\n\nAqui está uma pergunta: {input}'), additional_kwargs={})]), llm=ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002333B3155B0>, default_metadata=()), output_parser=StrOutputParser(), llm_kwargs={}),
 'Matematica': LLMChain(verbose=True, prompt=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromp

In [11]:
destinos = [f'{p["name"]}:{p['description']}' for p in prompt_infos]

destinos_str = ' \n'.join(destinos)

destinos_str

'Fisica:Ideal para responder perguntas de física \nMatematica:Ideal para responder perguntas de matemática \nHistoria:Ideal para responder perguntas de história'

In [12]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=  destinos_str)

print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
Fisica:Ideal para responder perguntas de física 
Matematica:Ideal para responder perguntas de matemát

In [13]:
router_template = PromptTemplate(
    template = router_template,
    input_varibles=['input'],
    output_parser=RouterOutputParser()
    )

router_chain  = LLMRouterChain.from_llm(chat, router_template, verbose=True)

In [14]:
default_prompt = ChatPromptTemplate.from_template('{input}')
default_chain = LLMChain(llm=chat, prompt=default_prompt, verbose=True)

In [17]:
chain = MultiPromptChain(router_chain = router_chain,
                         destination_chains=chains_destino,
                         default_chain=default_chain,
                         verbose=True)

chain.invoke({'input': 'O que é o  buraco negro?'})





> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
Fisica: {'input': 'O que é um buraco negro?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: Você é um professor de física muito inteligente.
Você é ótimo em responder perguntas sobre física de forma concisa
e fácil de entender.
Quando você não sabe a resposta para uma pergunta, você admite
que não sabe.

Aqui está uma pergunta: O que é um buraco negro?

> Finished chain.

> Finished chain.


{'input': 'O que é um buraco negro?',
 'text': 'Um buraco negro é uma região do espaço-tempo com uma gravidade tão forte que nada, nem mesmo a luz, pode escapar.  Ele é formado pelo colapso gravitacional de uma estrela muito massiva.'}

In [18]:
chain.invoke({'input': 'Quando foi a revolulção  industrial'})



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
Historia: {'input': 'Quando foi a revolução industrial?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: Você é um historiador muito bom.
Você tem um excelente conhecimento e compreensão de pessoas,
eventos e contextos de uma variedade de períodos históricos.
Você tem a capacidade de pensar, refletir, debater, discutir e
avaliar o passado. Você tem respeito pela evidência histórica
e a capacidade de usá-la para apoiar suas explicações
e julgamentos.

Aqui está uma pergunta: Quando foi a revolução industrial?

> Finished chain.

> Finished chain.


{'input': 'Quando foi a revolução industrial?',
 'text': 'Não existe uma data única e precisa para o início da Revolução Industrial. Foi um processo gradual e complexo que se desenvolveu ao longo de várias décadas, e até mesmo séculos, com diferentes aspectos emergindo em diferentes lugares e momentos.  Não se tratou de um evento repentino, mas de uma transformação profunda e duradoura.\n\nNo entanto, a maioria dos historiadores aponta para a **segunda metade do século XVIII na Grã-Bretanha** como o período de início da Revolução Industrial.  Mais especificamente,  a década de **1760 a 1780** é frequentemente citada como um período crucial, marcado por inovações tecnológicas fundamentais que impulsionaram mudanças econômicas e sociais significativas.\n\nÉ importante destacar alguns pontos:\n\n* **Inovações tecnológicas:**  A invenção e aperfeiçoamento de máquinas como a máquina a vapor, o tear mecânico e a máquina de fiar algodão foram cruciais. Essas inovações aumentaram significativa